In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
import numpy as np

data = np.random.randn(3,1,2,10) # 数据产生
mean = data.mean(axis = 3, keepdims=True)
mean.shape
x1 = data[:,:,[0,1],:]-mean
y1 = data[:,:,[1,0],:]-mean  # 矩阵翻转
coef = (x1*y1).sum(axis = 3, keepdims=True)/(10-1)
print(coef)

[[[[-0.35371701]
   [-0.35371701]]]


 [[[-0.58516381]
   [-0.58516381]]]


 [[[ 0.05689193]
   [ 0.05689193]]]]


In [6]:
coef.shape

(3, 1, 2, 1)

In [7]:
test_data = data[0].reshape(2,10)
print(np.cov(test_data))
test_data = data[1].reshape(2,10)
print(np.cov(test_data))
test_data = data[2].reshape(2,10)
print(np.cov(test_data))

[[ 0.37940968 -0.02780285]
 [-0.02780285  0.58783149]]
[[1.05488895 0.27507702]
 [0.27507702 0.8593024 ]]
[[ 1.60858621 -0.06614746]
 [-0.06614746  1.09033399]]


In [3]:
data=np.random.uniform(10,100,(2000,1,9,30))

# feat_nums = data.shape[2]
# list1 = list(range(feat_nums))
# num = generateC(list1)
# num_rev = []
# for l in num:
#     l1 = l.copy()
#     l1.reverse()
#     num_rev.append(l1)
# print(num,'\n',num_rev)



len(num)
    

NameError: name 'num' is not defined

In [8]:
class data_info:
    def __init__(self,data,stride):
        self.data = data
        self.data_length = data.shape[3]
        self.feat_num = data.shape[2]
#         self.num,self.num_rev = generate_Num_and_ReversedNum(feat_nums)
#         self.conv_feat = len(self.num)
#         self.step_list = get_step_list(data_length,stride)
    def generate_Num_and_ReversedNum(feat_nums):
        list1 = list(range(feat_nums))
        num = generateC(list1)
        num_rev = []
        for l in num:
            l1 = l.copy()
            l1.reverse()
            num_rev.append(l1)
        return num , num_rev
    def get_step_list():
        if data_length % stride == 0:
            step_list = list(range(0,data_length+stride,stride))
        elif data_length % stride<=5:
            mod = data_length % stride
            step_list = list(range(0,data_length-stride,stride))+[data_length]
        else:
            mod = data_length % stride
            step_list = list(range(0,data_length+stride-mod,stride))+[data_length]
        return step_list
one = data_info(data,10)
two = one.get_step_list()

TypeError: get_step_list() takes 0 positional arguments but 1 was given

In [15]:
stride = 10

# 为了cov 和 corr提取并行计算产生计算列表
def generateC(l1):
    if len(l1) == 1:
        return []
    v = [[l1[0],i] for i in l1[1:]]
    l1 = l1[1:]
    return v+generateC(l1)
def generate_Num_and_ReversedNum(feat_nums):
    list1 = list(range(feat_nums))
    num = generateC(list1)
    num_rev = []
    for l in num:
        l1 = l.copy()
        l1.reverse()
        num_rev.append(l1)
    return num , num_rev
def data_info(data,stride):
    if len(data.shape)!=4:
        raise Exception('Input data dimensions should be [N,C,H,W]')
    data_length = data.shape[3] # 30
    feat_num = data.shape[2] # 9
    """num:组合数对列表,num_rev:num的翻转列表"""
    num , num_rev = generate_Num_and_ReversedNum(feat_num)
    conv_feat = len(num)
    # 构建步长列表，如果数据长度不能整除，则取剩下长度，如果剩下长度小于5，则与上一步结合一起
    if data_length % stride == 0:
        step_list = list(range(0,data_length+stride,stride))
    elif data_length % stride<=5:
        mod = data_length % stride
        step_list = list(range(0,data_length-stride,stride))+[data_length]
    else:
        mod = data_length % stride
        step_list = list(range(0,data_length+stride-mod,stride))+[data_length]
    return data_length,feat_num,conv_feat,num,num_rev,step_list
def ts_cov4d(data,stride):
    '''计算4维数据的协方差'''
    '''data:[N,C,H,W],,W:price length,N:batch size'''
    if len(data.shape)!=4:
        raise Exception('Input data dimensions should be [N,C,H,W]')
    data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
    l = []
    #计算的过程中务必保持keepdims=True
    for i in range(len(step_list)-1):
        start = step_list[i]
        end = step_list[i+1]
        sub_data1 = data[:,:,num,start:end] # (2000, 1, 36, 2, 10)
        sub_data2 = data[:,:,num_rev,start:end]
        mean1 = sub_data1.mean(axis = 4,keepdims = True) # (2000, 1, 36, 2, 1)
        mean2 = sub_data2.mean(axis = 4,keepdims = True)
        spread1 = sub_data1 - mean1 # (2000, 1, 36, 2, 10)
        spread2 = sub_data2 - mean2
        cov = ((spread1*spread2).sum(axis = 4,keepdims = True)/(sub_data1.shape[4] - 1)).mean(axis = 3,keepdims = True) # (2000, 1, 36, 1, 1)
        l.append(cov)
    corr = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,1,conv_feat,len(step_list)-1) # (2000, 1, 36, 3)
    return torch.from_numpy(corr)
def ts_corr4d(data,stride):
    '''计算4维数据的相关系数'''
    '''data:[N,C,H,W],,W:price length,N:batch size'''
    if len(data.shape)!=4:
        raise Exception('Input data dimensions should be [N,C,H,W]')
    data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
    l = []
    for i in range(len(step_list)-1):
        start = step_list[i]
        end = step_list[i+1]
        sub_data1 = data[:,:,num,start:end]
        sub_data2 = data[:,:,num_rev,start:end]
        std1 = sub_data1.std(axis = 4,keepdims = True)
        std2 = sub_data2.std(axis = 4,keepdims = True)
        std = (std1*std2).mean(axis = 3,keepdims = True)
        l.append(std)
    std = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,1,conv_feat,len(step_list)-1)
    cov = ts_cov4d(data,num,num_rev,stride)
    fct = (sub_data1.shape[4]-1)/sub_data1.shape[4]
    return (cov/torch.from_numpy(std))*fct

In [16]:
cov = ts_corr4d(data,10)
cov.shape

TypeError: ts_cov4d() takes 2 positional arguments but 4 were given

In [29]:
data = np.random.uniform(10,100,(2000,1,9,30))#随机产生2000只股票的数据图片
ts_cov = ts_corr4d(data,num,num_rev,10)
print(ts_cov.shape)

torch.Size([2000, 1, 36, 3])
